In [1]:
# Import all of the things you need to import!
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem.porter import PorterStemmer
from sklearn.cluster import KMeans

pd.options.display.max_columns = 30
%matplotlib inline

# Homework 14 (or so): TF-IDF text analysis and clustering

Hooray, we kind of figured out how text analysis works! Some of it is still magic, but at least the **TF** and **IDF** parts make a little sense. Kind of. Somewhat.

No, just kidding, we're *professionals* now.

## Investigating the Congressional Record

The [Congressional Record](https://en.wikipedia.org/wiki/Congressional_Record) is more or less what happened in Congress every single day. Speeches and all that. A good large source of text data, maybe?

Let's pretend it's totally secret but we just got it leaked to us in a data dump, and we need to check it out. It was leaked from [this page here](http://www.cs.cornell.edu/home/llee/data/convote.html).

In [2]:
# If you'd like to download it through the command line...
!curl -O http://www.cs.cornell.edu/home/llee/data/convote/convote_v1.1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9607k  100 9607k    0     0  5201k      0  0:00:01  0:00:01 --:--:-- 5201k


In [3]:
# And then extract it through the command line...
!tar -zxf convote_v1.1.tar.gz

You can explore the files if you'd like, but we're going to get the ones from `convote_v1.1/data_stage_one/development_set/`. It's a bunch of text files.

In [4]:
# glob finds files matching a certain filename pattern
import glob

# Give me all the text files
paths = glob.glob('convote_v1.1/data_stage_one/development_set/*')
paths[:5]

['convote_v1.1/data_stage_one/development_set/052_400011_0327014_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_0327025_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_0327044_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_0327046_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_1479036_DON.txt']

In [5]:
len(paths)

702

So great, we have 702 of them. Now let's import them.

In [6]:
speeches = []
for path in paths:
    with open(path) as speech_file:
        speech = {
            'pathname': path,
            'filename': path.split('/')[-1],
            'content': speech_file.read()
        }
    speeches.append(speech)
speeches_df = pd.DataFrame(speeches)
speeches_df.head()

,content,filename,pathname
0,"mr. chairman , i thank the gentlewoman for yie...",052_400011_0327014_DON.txt,convote_v1.1/data_stage_one/development_set/05...
1,"mr. chairman , i want to thank my good friend ...",052_400011_0327025_DON.txt,convote_v1.1/data_stage_one/development_set/05...
2,"mr. chairman , i rise to make two fundamental ...",052_400011_0327044_DON.txt,convote_v1.1/data_stage_one/development_set/05...
3,"mr. chairman , reclaiming my time , let me mak...",052_400011_0327046_DON.txt,convote_v1.1/data_stage_one/development_set/05...
4,"mr. chairman , i thank my distinguished collea...",052_400011_1479036_DON.txt,convote_v1.1/data_stage_one/development_set/05...


In class we had the `texts` variable. For the homework can just do `speeches_df['content']` to get the same sort of list of stuff.

**Take a look at the contents of the first 5 speeches**

In [7]:
for item in speeches_df['content'][:5]:
    print(item[:140], "\n")

mr. chairman , i thank the gentlewoman for yielding me this time . 
my good colleague from california raised the exact and critical point .  

mr. chairman , i want to thank my good friend from california ( mr. rohrabacher ) xz4003430 . 
i will always remember that day , as we all w 

mr. chairman , i rise to make two fundamental points before we proceed to vote on this . 
the two points are these : this resolution does no 

mr. chairman , reclaiming my time , let me make two final points : one , the majority party must understand this : if you are at a republica 

mr. chairman , i thank my distinguished colleague , and i appreciate his leadership on this issue . 
the gentleman from california ( mr. roh 



# Doing our analysis

Use the `sklearn` package and a plain boring `CountVectorizer` to get a list of all of the tokens used in the speeches. If it won't list them all, that's ok! Make a dataframe with those terms as columns.

**Be sure to include English-language stopwords**

In [8]:
count_vectorizer = CountVectorizer(stop_words='english')
X = count_vectorizer.fit_transform(speeches_df['content'])
X

<702x9106 sparse matrix of type '<class 'numpy.int64'>'
	with 56106 stored elements in Compressed Sparse Row format>

In [9]:
X_df = pd.DataFrame(X.toarray(), columns=count_vectorizer.get_feature_names())

In [10]:
X_df.head(10)

,000,00007,018,050,092,10,100,106,107,108,108th,109th,10th,11,110,...,yields,york,yorkers,young,younger,youngsters,youth,yuan,zero,zeroing,zeros,zigler,zirkin,zoe,zoellick
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Okay, it's **far** too big to even look at. Let's try to get a list of features from a new `CountVectorizer` that only takes the top 100 words.

In [11]:
count_vectorizer = CountVectorizer(stop_words='english', max_features=100)
X = count_vectorizer.fit_transform(speeches_df['content'])
X

<702x100 sparse matrix of type '<class 'numpy.int64'>'
	with 11088 stored elements in Compressed Sparse Row format>

In [12]:
pd.DataFrame(X.toarray(), columns=count_vectorizer.get_feature_names()).head()

,000,11,act,allow,amendment,america,american,amp,association,balance,based,believe,bipartisan,chairman,children,...,teachers,thank,think,time,today,trade,united,urge,vote,want,way,work,year,years,yield
0,0,1,3,0,0,0,3,0,0,0,0,1,0,3,0,...,0,1,3,3,2,0,1,0,0,1,1,0,0,0,1
1,0,0,1,1,1,0,0,0,0,1,0,0,0,2,0,...,0,1,0,2,2,0,0,0,1,1,3,0,1,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,0,0,0,0,0,1,0,0,0,1,0,0,0,2,0,...,0,0,0,2,0,0,1,0,1,1,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,...,0,1,0,1,0,0,0,0,2,0,0,0,0,0,2


Now let's push all of that into a dataframe with nicely named columns.

In [13]:
X_df = pd.DataFrame(X.toarray(), columns=count_vectorizer.get_feature_names())

Everyone seems to start their speeches with "mr chairman" - how many speeches are there total, and many don't mention "chairman" and how many mention neither "mr" nor "chairman"?

In [14]:
no_chairman = X_df[X_df['chairman'] == 0]['chairman'].count()
no_chairman_no_mr = X_df[(X_df['chairman'] == 0) & (X_df['mr'] == 0)]['chairman'].count()
print("In a total of", len(X_df), "speeches,", no_chairman, "don't mention “chairman” and", no_chairman_no_mr, "mention neither “mr” nor “chairman”.")

In a total of 702 speeches, 250 don't mention “chairman” and 76 mention neither “mr” nor “chairman”.


What is the index of the speech thank is the most thankful, a.k.a. includes the word 'thank' the most times?

In [15]:
print("The index of this speech is", X_df['thank'].idxmax())

The index of this speech is 577


If I'm searching for `China` and `trade`, what are the top 3 speeches to read according to the `CountVectoriser`?

In [16]:
china_trade = X_df.sort_values(by=['china', 'trade'], ascending=[0, 0])[['china', 'trade']].head(3)
print("These three speeches have the indexes ", *list(china_trade.index))
china_trade

These three speeches have the indexes  379 399 345


,china,trade
379,29,63
399,27,9
345,16,11


Now what if I'm using a `TfidfVectorizer`?

In [17]:
def simple_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    return words
tfidf_vectorizer = TfidfVectorizer(stop_words='english', tokenizer=simple_tokenizer, use_idf=False, norm='l1')
X = tfidf_vectorizer.fit_transform(speeches_df['content'])
TF_pd = pd.DataFrame(X.toarray(), columns=tfidf_vectorizer.get_feature_names())

china_trade = TF_pd.sort_values(by=['china', 'trade'], ascending=[0, 0])[['china', 'trade']].head(3)
print("The three top speeches have the indexes ", *list(china_trade.index))
china_trade

The three top speeches have the indexes  345 340 315


,china,trade
345,0.078818,0.054187
340,0.057377,0.008197
315,0.055000,0.035000


**What's the content of the speeches?** Here's a way to get them:

In [18]:
# index 0 is the first speech, which was the first one imported.
paths[0]

'convote_v1.1/data_stage_one/development_set/052_400011_0327014_DON.txt'

In [19]:
# Pass that into 'cat' using { } which lets you put variables in shell commands
# that way you can pass the path to cat
!cat {paths[0]}

mr. chairman , i thank the gentlewoman for yielding me this time . 
my good colleague from california raised the exact and critical point . 
the question is , what happens during those 45 days ? 
we will need to support elections . 
there is not a single member of this house who has not supported some form of general election , a special election , to replace the members at some point . 
but during that 45 days , what happens ? 
the chair of the constitution subcommittee says this is what happens : martial law . 
we do not know who would fill the vacancy of the presidency , but we do know that the succession act most likely suggests it would be an unelected person . 
the sponsors of the bill before us today insist , and i think rightfully so , on the importance of elections . 
but to then say that during a 45-day period we would have none of the checks and balances so fundamental to our constitution , none of the separation of powers , and that the presidency would be filled by an unel

**Now search for something else!** Another two terms that might show up. `elections` and `chaos`? Whatever you thnik might be interesting.

In [20]:
numbers = list(range(0, 10))
numbers = list(map(str, numbers))
words_list = [i for i in list(TF_pd.columns) if i[0] not in numbers]
words_list

['-',
 '--',
 '-central',
 '-china',
 '-women',
 'aaron',
 'aba',
 'abandon',
 'abandoned',
 'abandoning',
 'abcs',
 'abet',
 'abhorrent',
 'abide',
 'abides',
 'abiding',
 'abilities',
 'ability',
 'able',
 'ably',
 'abolish',
 'abraham',
 'abridgement',
 'abroad',
 'abrogation',
 'absence',
 'absent',
 'absentee',
 'absolutely',
 'absolve',
 'absorb',
 'absurd',
 'abundance',
 'abundant',
 'abuse',
 'abused',
 'abuses',
 'abusing',
 'abusive',
 'abysmal',
 'academic',
 'academically',
 'academics',
 'academy',
 'accede',
 'accelerated',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'accepts',
 'access',
 'accessible',
 'accessing',
 'accession',
 'accessories',
 'accident',
 'accidents',
 'acclaimed',
 'accommodate',
 'accommodated',
 'accommodating',
 'accompanies',
 'accompanying',
 'accomplish',
 'accomplished',
 'accomplishes',
 'accomplishment',
 'accordance',
 'according',
 'accordingly',
 'account',
 'accountability',
 'accountable',
 'accountant',
 'acc

In [21]:
chaos = TF_pd.sort_values(by=['awfully', 'bacterial'], ascending=[0, 0])[['awfully', 'bacterial']].head(3)
print("The three top speeches have the indexes ", *list(chaos.index))
chaos



The three top speeches have the indexes  392 204 0


,awfully,bacterial
392,0.004132,0.000000
204,0.000000,0.009174
0,0.000000,0.000000


In [22]:
gun_bomb = TF_pd.sort_values(by=['gun', 'bomb'], ascending=[0, 0])[['gun', 'bomb']].head(3)
print("The three top speeches have the indexes ", *list(gun_bomb.index))
gun_bomb




The three top speeches have the indexes  644 661 133


,gun,bomb
644,0.001876,0.000
661,0.000553,0.000
133,0.000000,0.004


# Enough of this garbage, let's cluster

Using a **simple counting vectorizer**, cluster the documents into **eight categories**, telling me what the top terms are per category.

Using a **term frequency vectorizer**, cluster the documents into **eight categories**, telling me what the top terms are per category.

Using a **term frequency inverse document frequency vectorizer**, cluster the documents into **eight categories**, telling me what the top terms are per category.

**`CountVectorizer()`:** Convert a collection of text documents to a matrix of token counts

**`TfidfVectorizer(use_idf=False)`:** Convert a collection of raw documents to a matrix of TF-IDF features. Equivalent to CountVectorizer followed by TfidfTransformer.

**`TfidfVectorizer(use_idf=True)` (default):** Enable inverse-document-frequency reweighting.

In [23]:
countingVectorizer = CountVectorizer(tokenizer=simple_tokenizer, stop_words='english')
TF_Vectorizer = TfidfVectorizer(use_idf=False, tokenizer=simple_tokenizer, stop_words='english')
TF_IDF_Vectorizer = TfidfVectorizer(use_idf=True, tokenizer=simple_tokenizer, stop_words='english')
Vectorizer_list = [countingVectorizer, TF_Vectorizer, TF_IDF_Vectorizer]
Vectorizer_names = ['', 'simple counting vectorizer', 'term frequency vectorizer', 'term frequency IDF vectorizer']

In [24]:
count = 1
for vectorizer in Vectorizer_list:
    print("\n[" + str(count) + "]", Vectorizer_names[count])

    X = vectorizer.fit_transform(speeches_df['content'])
    number_of_clusters = 8
    km = KMeans(n_clusters=number_of_clusters)
    km.fit(X)
    
    order_centroids = km.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(number_of_clusters):
        top_five_words = [terms[ind] for ind in order_centroids[i, :10]]
        print("Cluster {}: {}".format(i, ' '.join(top_five_words)))
        
    count += 1


[1] simple counting vectorizer
Cluster 0: mr time house s chairman legislation people states elections amendment
Cluster 1: head start religious rights civil program discrimination protections amendment programs
Cluster 2: mr chairman gentleman amendment time yield speaker s committee house
Cluster 3: nbsp amp gt p lt trade -- s united states
Cluster 4: rule 11 rules federal h r 420 sanctions judicial litigation
Cluster 5: association national restaurant contractors chamber amp electrical commerce chapter american
Cluster 6: start head children program amendment mr programs school s chairman
Cluster 7: trade china s american cafta church mr u vote people

[2] term frequency vectorizer
Cluster 0: mr chairman amendment time gentleman s yield horses epa offer
Cluster 1: mr yield chairman gentleman minutes 2 1 speaker committee member
Cluster 2: start head children program amendment mr chairman programs s religious
Cluster 3: time mr balance chairman yield reserve speaker gentleman remain

**Which one do you think works the best?**

The TF-IDF is definitely the most efficient one, at least in this case!

# Harry Potter time

I have a scraped collection of Harry Potter fanfiction at https://github.com/ledeprogram/courses/raw/master/algorithms/data/hp.zip.

I want you to read them in, vectorize them and cluster them. Use this process to find out **the two types of Harry Potter fanfiction**. What is your hypothesis?

`curl -LO`

In [25]:
!curl -LO https://github.com/ledeprogram/courses/raw/master/algorithms/data/hp.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   149  100   149    0     0    132      0  0:00:01  0:00:01 --:--:--   132
100 9226k  100 9226k    0     0  1649k      0  0:00:05  0:00:05 --:--:-- 2129k


In [26]:
#!unzip hp.zip
paths_potter = glob.glob('hp/*')
paths_potter[:3]

['hp/10001898.txt', 'hp/10004131.txt', 'hp/10004927.txt']

In [27]:
potter_texts = []
for path in paths_potter:
    with open(path) as speech_file:
        text = {
            'pathname': path,
            'filename': path.split('/')[-1],
            'content': speech_file.read()
        }
    potter_texts.append(text)
potter_df = pd.DataFrame(potter_texts)
potter_df.head(2)

,content,filename,pathname
0,Prologue: The MissionDisclaimer: All character...,10001898.txt,hp/10001898.txt
1,BlackDisclaimer: I do not own Harry PotterAuth...,10004131.txt,hp/10004131.txt


In [28]:
#1
vectorizer = TfidfVectorizer(use_idf=True, tokenizer=simple_tokenizer, stop_words='english')
X = vectorizer.fit_transform(potter_df['content'])

#2
number_of_clusters = 2
km = KMeans(n_clusters=number_of_clusters)
km.fit(X)

#3
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :5]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

#4
results = pd.DataFrame()
results['text'] = potter_df['content']
results['category'] = km.labels_
results.head(10)

Cluster 0: t s lily james sirius
Cluster 1: harry hermione t s draco


,text,category
0,Prologue: The MissionDisclaimer: All character...,0
1,BlackDisclaimer: I do not own Harry PotterAuth...,0
2,"Chapter 1""I'm pregnant.""""""""Mum please say some...",1
3,"Author's Note: Hey, just so you know, this is ...",0
4,Disclaimer: I do not own Harry Potter and frie...,0
5,Disclaimer: I don't own any character in the H...,0
6,DISCLAIMER: I don't own Harry Potter and its c...,1
7,Katherine Rose-TylerChapter One: the Introduct...,0
8,I am no longer that shy little boy anymore.I w...,1
9,Happy New year! *throws confetti*I've really b...,0


# The two types of Harry Potter fanfiction
* story of Potter's parents
* story of Harry, Hermione and Draco